In [1]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv2D , MaxPooling2D, UpSampling2D, Conv2DTranspose, BatchNormalization, Activation, Add
from tensorflow.keras.layers import ZeroPadding2D, Input, AveragePooling2D, Flatten, Dense, Dropout, concatenate
from matplotlib import pyplot as plt

##################
# IMPORTS
##################

import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import numpy as np
import tensorflow 
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)
import os, sys
module_path = os.path.abspath(os.path.join('../../support/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
SAVE_MODEL = "classification-vgg16-3"
    
#https://towardsdatascience.com/galaxy-zoo-classification-with-keras-219184aff581    
    
import DatasetUtilsTifF as dsutils

In [2]:
img_width=64; img_height=64
IMG_PATH = "C:/Projects/VenusDenoise/dataset/cases/64/0100_1000/"
TRAIN_CLASSES_PATH = "C:/Projects/VenusDenoise/out_tests/0100_1000-64-train7.csv"
VALID_CLASSES_PATH = "C:/Projects/VenusDenoise/out_tests/train_classes/0100_1000-64-7.csv"

batch_size=16
class RadianceLimits:
    pass
radiance_limits = RadianceLimits()
radiance_limits.noisy_min = 0
radiance_limits.noisy_max = 0.0898
radiance_limits.nitid_min = 0
radiance_limits.nitid_max = 0.3248

In [4]:
train_files, train_classes, train_images = \
        dsutils.readDatasetClassification(IMG_PATH, img_width, img_height, radiance_limits, TRAIN_CLASSES_PATH)

val_files, val_classes, val_images = \
    dsutils.readDatasetClassification(IMG_PATH + "validation/", img_width, img_height, radiance_limits, VALID_CLASSES_PATH)

Not found:VI0112_00_00_10_


SystemExit: -1

C:\Users\rdelasheras\Anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
train_images = np.reshape(train_images, (len(train_images), img_height, img_width, 1))  
val_images= np.reshape(val_images, (len(val_images), img_height, img_width, 1))  

#train_classes = train_classes.squeeze()
#valid_classes = valid_classes.squeeze()

train_classes= np.reshape(train_classes, (len(train_classes), 1))  
val_classes= np.reshape(val_classes, (len(val_classes), 1))  

In [ ]:
train_classes = tf.keras.utils.to_categorical( train_classes, num_classes=None, dtype='float32')
val_classes = tf.keras.utils.to_categorical( val_classes, num_classes=None, dtype='float32')

In [ ]:
input_layer = Input((img_width, img_height,1))

conv1 = Conv2D(64, (3,3), activation='relu', padding='same')(input_layer)
conv1 = Conv2D(64, (3,3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2,2))(conv1)

conv2 = Conv2D(128, (3,3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, (3,3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2,2))(conv2)

conv3 = Conv2D(256, (3,3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, (3,3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D((2,2))(conv3)

conv4 = Conv2D(512, (3,3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, (3,3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D((2,2))(conv4)

conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(conv5)
pool5 = MaxPooling2D((2,2))(conv5)

dense1 = Dense(4096, activation='relu')(pool5)
dense1 = Dropout(0.5)(dense1)
#dense2 = Dense(4096, activation='relu')(dense1)
#dense2 = Dropout(0.5)(dense2)

flatten = Flatten()(dense1)

output_layer = Dense(4, activation = 'softmax')(flatten)

model = Model(input_layer, output_layer)

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Nadam
model.compile(optimizer=Nadam(learning_rate=0.00001), loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
hist = model.fit(train_images, train_classes, 
                        epochs=200,
                        batch_size=16, 
                        verbose=1, 
                        validation_data=(val_images, val_classes))


In [ ]:
##################
# REPORTS
##################
import ReportsK as reports
reports.plotHistory( hist )

In [ ]:

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])

In [ ]:
model.save(os.path.abspath(os.path.join('../../../saves/', "classification-vgg16-3")), model)